<a href="https://colab.research.google.com/github/xwu391/6258-DIP/blob/master/Conv_AE3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import torch
import torchvision
from  torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from  torchvision import datasets, transforms
from  sklearn.decomposition.pca import PCA
import cv2

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive


In [ ]:
class Conv_AE(nn.Module):
  def __init__(self):
    super(Conv_AE, self).__init__()
    self.Encoder_Net = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(inplace=True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(inplace=True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(inplace=True),

        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(in_channels=32, out_channels=4, kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(inplace=True),
    )

    self.Decoder_Net = nn.Sequential(
        nn.ConvTranspose2d(in_channels=4, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(inplace=True),
        nn.BatchNorm2d(32),

        nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=2, stride=2),
        nn.LeakyReLU(inplace=True),
        nn.BatchNorm2d(32),

        nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=2, stride=2),
        nn.LeakyReLU(inplace=True),
        nn.BatchNorm2d(32),

        nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=2, stride=2),
        nn.LeakyReLU(inplace=True),
        nn.BatchNorm2d(32),

        nn.ConvTranspose2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
        nn.LeakyReLU(inplace=True),
        nn.BatchNorm2d(32),

        nn.Conv2d(in_channels=32, out_channels=1, kernel_size=1, stride=1),
        nn.Sigmoid()
    )

  def Encoder(self, input_img):
    x = self.Encoder_Net(input_img)
  #  x = self.fc1(x)
    return x

  def Decoder(self, input_img):
  #  x = self.fc2(input_img)
    x = self.Decoder_Net(input_img)
    return x
    
  def forward(self, input_img):
    code = self.Encoder(input_img)
    recover = self.Decoder(code)
    return recover, code
  

In [ ]:
Pre_transfrom = transforms.Compose([
        transforms.Resize((128,128)),
        transforms.Grayscale(),
        transforms.ToTensor(),
    ])

train_dataset = datasets.MNIST(root='./data/', train=True, transform=Pre_transfrom, download=True)
#test_dataset = datasets.MNIST(root='./data/', train=False, transform=Pre_transfrom)

train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=200, num_workers=20, drop_last=True)

conv_AE = Conv_AE().cuda()
loss_function = nn.MSELoss()
optimizer = optim.SGD(conv_AE.parameters(), lr=0.001, momentum=0.9)


for epoch in range(50):
  print("Epoch %d" % epoch)

  for i, (images, _) in enumerate(train_loader):
    images = Variable(images).cuda()
    recover, _ = conv_AE(images)
        
    optimizer.zero_grad()
    loss = loss_function(recover, images)
    loss.backward()
    optimizer.step()        
        
  print("Loss = %.5f" % loss.item()) 
  if (epoch%20==0):
    torch.save(conv_AE, 'CAE3-3.pkl')

print('Finished Training')

Epoch 0
Loss = 0.05177
Epoch 1


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

conv_AE = torch.load('CAE3-3.pkl')
Pre_transfrom = transforms.Compose([
        transforms.Resize((128,128)),
        transforms.Grayscale(),
        transforms.ToTensor(),
    ])

test_dataset = datasets.ImageFolder(root='test/', transform=Pre_transfrom)

test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False, batch_size=1, num_workers=1, drop_last=True)

for i, (images, _) in enumerate(test_loader):
    images = Variable(images).cuda()
    recover, code = conv_AE(images)
    torchvision.utils.save_image(recover.data, 'Conv1' + str(i) + '.png')

In [ ]:
pip install dippykit

     |████████████████████████████████| 61kB 3.7MB/s 
  Created wheel for dippykit: filename=dippykit-2.0.3-cp36-none-any.whl size=58952 sha256=6b24f45fb4e5018e5da058812a067bb30d04f0b3858a89b7cd6786fb9bc90cb7
  Stored in directory: /root/.cache/pip/wheels/23/42/9c/a692a4fe24e12db06f80e6e86dcc79cfe784ad832e9a5e96ff
Successfully built dippykit


In [ ]:
import dippykit as dip
code1 = torch.mul(code, 1000000).reshape(16*16*4)
code1 = torch.round(code1).cpu().detach().numpy()
code2 = np.zeros(256*4, dtype=int);
for i in range(256*4):
  code2[i] = int(code1[i])

img_en, stream_length, _, _ = dip.huffman_encode(code2)
print(stream_length)


9397
